In [28]:
import datetime
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Will plot in separate pop-up window
# This allows for dynamic plots in matplotlib
%matplotlib qt

# 0. IPCC Color Map

- Create a color map based on the IPCC visual style guide (https://www.ipcc.ch/site/assets/uploads/2019/04/IPCC-visual-style-guide.pdf)
    - Temperature section, page 10

In [13]:
# Set RGB values (using 11 color scheme for smoothest gradient)
red1 = [103, 0, 31]
red2 = [178, 24, 43]
red3 = [214, 96, 77]
red4 = [244, 165, 130]
red5 = [253, 219, 199]
white = [247, 247, 247]
blue5 = [209, 229, 240]
blue4 = [146, 197, 222]
blue3 = [67, 147, 195]
blue2 = [33, 102, 172]
blue1 = [5, 48, 97]

# Normalize values from [0, 256] to [0, 1]
color_list = [blue1, blue2, blue3, blue4, blue5, white, red5, red4, red3, red2, red1]
color_tuples = []
for color in color_list:
    color_tuples.append([x/256 for x in color])

# Create cmap for later plotting
IPCC_cmap = colors.LinearSegmentedColormap.from_list('IPCC_RGB', color_tuples)

# 1. Dynamic Temperature Anomaly Timeseries Plot

- Using Global-mean monthly, seasonal, and annual means data from https://data.giss.nasa.gov/gistemp/
- File can be found under 'Table of Global and Hemispheric Monthly Means and Zonal Annual Means'

In [10]:
# Read in GISTEMP data as dataframe, skip header, set year as index
gistemp_file = 'GLB.Ts+dSST.csv'
df_full = pd.read_csv(gistemp_file, header=1)
df_full = df_full.set_index('Year')

# Drop non-monthly columns
df = df_full.iloc[:, :12]

# Drop 2023 (incomplete)
df = df.head(len(df) - 1)

# Convert to floats
df = df.astype(float)

In [17]:
# Function for plotting all data points for a given month
def monthly_timeseries(column='Jan'):
    
    # Initialize lists/counter, set min/max values for colorbar and number of points
    temp_list = []
    year_list = []
    counter = 0
    num_points = len(df) - 1
    max_color = 1.25
    min_color = -max_color

    # Loop through all rows in dataframe
    while True:

        # Gather values cumulatively
        temp = df[column].iloc[counter]
        year_list.append(int(df.index[counter]))
        temp_list.append(temp)

        # Line plot
        plt.rcParams["figure.figsize"] = (10, 10)
        plt.plot(year_list, temp_list, color='black')
        
        # Horizontal line for baseline temperature
        plt.axhline(y=0, color='black', linestyle='dashed')
        
        # Vertical line for anomaly baseline years
        plt.axvline(x=1951, color='black', linestyle='dashed')
        plt.axvline(x=1980, color='black', linestyle='dashed')

        # Set vertical bounds (with padding)
        ymin = min(df[column]) * 1.1
        ymax = max(df[column]) * 1.1
        ymin = round(ymin, 1) - 0.05
        ymax = round(ymax, 1) + 0.05
        
        # Scatter plot     
        plt.scatter(year_list, temp_list, c=temp_list, cmap=IPCC_cmap)
        plt.clim(min_color, max_color)

        # Formatting
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Temperature Anomaly (\xb0C)', fontsize=14)
        plt.ylim(ymin, ymax)
        title = 'GISTEMP Global Temperature Anomaly'
        title += ' ('  + column +')'                    
        plt.title(title, fontsize=18)
        plt.text(x=1952, y=ymax*0.8, s='Time period \nused for \nbaseline\nanomaly\n1951-1980')
        plt.draw()
        plt.grid()
        plt.colorbar(boundaries = np.arange(ymin, ymax, 0.05),
                     ticks=np.linspace(min_color, max_color, 11)).set_label('\xb0C', fontsize=14)
        
        # Brief pause
        plt.pause(0.05)

        # Clear plot to make room for next one
        if counter < num_points:
            plt.clf()
            
        # If last plot, stop plotting
        else:
            break

        # Increment counter
        counter += 1

In [20]:
# Similar to the above plot, but including every monthly data point
# (Note that plotting this takes 12 times longer than the monthly plot)
def overall_timeseries():

    # Get a list of all monthly values
    values = []
    dates = []

    # Create list of strings for each month
    month_list = []
    for i in range(12):
        month_num = str(i+1)
        if len(month_num) < 2:
            month_num = '0' + month_num
        month_list.append(month_num)

    # Collect datetime and values for each entry in dataframe
    for i in range(len(df)):
        year = df.iloc[i].name
        months = df.iloc[i].index
        val_i = df.iloc[i].values
        month_counter = 0
        for j in range(len(val_i)-1):
            values.append(val_i[j])
            date_str = str(year) + '-' + month_list[month_counter]
            dt = datetime.datetime.strptime(date_str, '%Y-%m')
            dates.append(dt)
            month_counter += 1

    # Initialize lists/counter, set min/max values for colorbar and number of points
    temp_list = []
    year_list = []
    counter = 0
    num_points = len(values) - 1
    max_color = 1.25
    min_color = -max_color

    # Loop through each row in dataframe
    while True:

        # Gather values cumulatively
        temp_list.append(values[counter])
        year_list.append(dates[counter])

        # Line plot
        plt.rcParams["figure.figsize"] = (10, 10)
        plt.plot(year_list, temp_list, color='black')

        # Horizontal line for baseline temperature anomaly
        plt.axhline(y=0, color='black', linestyle='dashed')
        
        # Set vertical bounds
        ymin = min(df.min()) * 1.1
        ymax = max(df.max()) * 1.1
        ymin = round(ymin, 1) - 0.05
        ymax = round(ymax, 1) + 0.05
        
        # Scatter plot
        plt.scatter(year_list, temp_list, c=temp_list, cmap=IPCC_cmap)
        plt.clim(min_color, max_color)

        # Formatting
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Temperature Anomaly (\xb0C)', fontsize=14)
        ymin = min(values) * 1.1
        ymax = max(values) * 1.1
        plt.ylim(ymin, ymax)
        title = 'GISTEMP Global Temperature Anomaly'
        plt.title(title, fontsize=18)
        plt.draw()
        plt.grid()
        plt.colorbar(boundaries = np.arange(ymin, ymax, 0.05),
                     ticks=np.linspace(min_color, max_color, 11)).set_label('\xb0C', fontsize=14)

        # Very small pause
        plt.pause(10**(-10))

        # Clear plot to make room for next one
        if counter < num_points:
            plt.clf()
            
        # If last plot, stop plotting
        else:
            break

        # Increment counter
        counter += 1

In [21]:
monthly_timeseries('Jan')

In [22]:
overall_timeseries()

# 2. Dynamic Spatial Temperature Anomaly Plot

In [29]:
# Read in GHCN data, open dataset using xarray
ghcn_file = 'gistemp1200_GHCNv4_ERSSTv5.nc'
ds = xr.open_dataset(ghcn_file)

# Collect dates, print number of data points
dates = ds['time'].values
print(f'Total number of dates: {len(dates)}')

Total number of dates: 1723


In [30]:
# Create dictionary for plotting individual months
month_dict = {}
month_dict['Jan'] = 0
month_dict['Feb'] = 1
month_dict['Mar'] = 2
month_dict['Apr'] = 3
month_dict['May'] = 4
month_dict['Jun'] = 5
month_dict['Jul'] = 6
month_dict['Aug'] = 7
month_dict['Sep'] = 8
month_dict['Oct'] = 9
month_dict['Nov'] = 10
month_dict['Dec'] = 11

In [31]:
# Function for plotting dynamic temperature data for each month
# (plotting all points would take too long)
def spatial_temp_plot(month='Jan'):

    # Collect all dates for given month
    month_dates = []
    for i in range(len(dates)):
        if i % 12 == month_dict[month]:
            month_dates.append(dates[i])

    # Initialize figure
    fig = plt.figure(figsize=(10,6))

    # Set counter and min/max values
    counter = 0
    max_val = 2
    min_val = -max_val
    num_points = len(month_dates) - 1

    # Loop through all dates
    while True:

        # Plot temperature anomaly for each month
        data = ds['tempanomaly'].sel(time=month_dates[counter])
        ax = plt.axes(projection=ccrs.Robinson())
        ax.coastlines()
        ax.gridlines()
        data.plot.pcolormesh(ax=ax,
                  cmap=IPCC_cmap,
                  transform=ccrs.PlateCarree(),
                  cbar_kwargs={'orientation':'horizontal', 'pad':0.1},
                  vmin=min_val,
                  vmax=max_val)
        plt.draw()
        date = str(data.time.values).split('T')[0]
        title = 'GISTEMP Global Temperature Anomaly\n' + date
        plt.title(title)
        plt.pause(10**(-3))

        # Clear plot to make room for next one
        if counter < num_points:
            plt.clf()

        # Stop plotting if last date reached
        else:
            break

        # Increment counter
        counter += 1

In [32]:
spatial_temp_plot('Apr')

# 3. Table for Zonal Annual Means

- Note: ZonAnn.Ts+dSST.csv is structured as 3 tables stacked on top of each other
    - This can be observed by viewing the text file (ZonAnn.Ts+dSST.txt) on the same page on the GISS website

In [76]:
# Show all rows
pd.set_option('display.max_rows', None)

# Create dataframe for all zonal annual means (AIRS v6)
zam_file = 'ZonAnn.Ts+dSST.csv'
df_AIRS_v6 = pd.read_csv(zam_file, header=1, nrows=20)

# Gather title (AIRS v6)
AIRS_v6_title = pd.read_csv(zam_file, nrows=1).columns[0]

# Print title and dataframe
print(AIRS_v6_title)
df_AIRS_v6

Annual Temperature Anomalies (deg C) AIRS v6 vs. 2007-2016


,Year,Glob,N.Hemi,S.Hemi,24N-90N,24S-24N,90S-24S,64N-90N,44N-64N,24N-44N,EQU-24N,24S-EQU,44S-24S,64S-44S,90S-64S
0,2003,-0.134,-0.293,0.026,-0.465,-0.006,0.028,-1.184,-0.248,-0.369,-0.035,0.022,-0.126,0.164,0.220
1,2004,-0.181,-0.263,-0.099,-0.391,-0.119,-0.054,-1.303,-0.238,-0.188,-0.072,-0.165,-0.094,0.026,-0.095
2,2005,-0.021,-0.066,0.024,-0.121,0.024,0.018,-0.292,0.123,-0.227,0.017,0.032,-0.145,0.034,0.478
3,2006,-0.080,-0.080,-0.080,-0.090,-0.107,-0.034,-0.191,-0.070,-0.070,-0.066,-0.148,-0.090,-0.038,0.142
4,2007,-0.063,-0.050,-0.076,0.011,-0.133,-0.044,0.022,0.155,-0.089,-0.142,-0.124,-0.124,-0.111,0.331
5,2008,-0.181,-0.223,-0.138,-0.187,-0.250,-0.082,-0.601,-0.021,-0.161,-0.277,-0.223,-0.034,-0.031,-0.328
6,2009,-0.033,-0.110,0.044,-0.228,0.051,0.050,-0.322,-0.434,-0.059,0.067,0.036,-0.004,0.098,0.117
7,2010,0.007,0.058,-0.045,0.021,0.057,-0.075,0.259,-0.074,0.005,0.114,-0.000,-0.102,0.091,-0.324
8,2011,-0.096,-0.127,-0.066,-0.080,-0.263,0.110,0.198,-0.125,-0.143,-0.197,-0.329,-0.010,0.095,0.497
9,2012,-0.061,-0.086,-0.035,-0.054,-0.086,-0.034,0.240,-0.190,-0.061,-0.134,-0.037,-0.056,-0.004,-0.027


In [77]:
# Create dataframe for all zonal annual means (AIRS v7)
df_AIRS_v7 = pd.read_csv(zam_file, header=23, nrows=20)

# Gather title (AIRS v7)
AIRS_v7_title = pd.read_csv(zam_file, header=22, nrows=1).columns[0]

# Print title and dataframe
print(AIRS_v7_title)
df_AIRS_v7

Annual Temperature Anomalies (deg C) AIRS v7 vs. 2007-2016


,Year,Glob,N.Hemi,S.Hemi,24N-90N,24S-24N,90S-24S,64N-90N,44N-64N,24N-44N,EQU-24N,24S-EQU,44S-24S,64S-44S,90S-64S
0,2003,0.031,-0.122,0.183,-0.318,0.189,0.168,-0.970,-0.030,-0.292,0.171,0.206,-0.013,0.385,0.277
1,2004,-0.066,-0.146,0.014,-0.288,0.019,0.042,-1.365,-0.047,-0.089,0.067,-0.028,-0.005,0.174,-0.081
2,2005,0.073,0.040,0.105,-0.002,0.111,0.096,-0.170,0.301,-0.148,0.104,0.118,-0.086,0.145,0.543
3,2006,-0.032,-0.038,-0.026,-0.048,-0.068,0.032,-0.211,0.039,-0.051,-0.022,-0.113,-0.038,0.074,0.161
4,2007,-0.029,-0.016,-0.042,0.055,-0.118,0.006,0.095,0.200,-0.055,-0.121,-0.114,-0.108,-0.021,0.404
5,2008,-0.171,-0.212,-0.129,-0.167,-0.247,-0.073,-0.529,0.004,-0.161,-0.279,-0.214,-0.058,0.026,-0.314
6,2009,-0.061,-0.195,0.074,-0.357,0.032,0.113,-0.712,-0.511,-0.136,0.047,0.016,0.002,0.176,0.319
7,2010,-0.001,0.038,-0.040,0.001,0.038,-0.055,0.205,-0.054,-0.030,0.093,-0.018,-0.088,0.151,-0.365
8,2011,-0.090,-0.125,-0.056,-0.078,-0.260,0.125,0.194,-0.100,-0.154,-0.195,-0.326,-0.017,0.164,0.470
9,2012,-0.063,-0.076,-0.050,-0.031,-0.097,-0.051,0.320,-0.163,-0.059,-0.144,-0.049,-0.046,-0.005,-0.156


In [78]:
# Create dataframe for all zonal annual means (GHCNv4/ERSSTv5)
df_GHCN_v4 = pd.read_csv(zam_file, header=45, nrows=20)

# Gather title (GHCNv4/ERSSTv5)
GHCN_v4_title = pd.read_csv(zam_file, header=44, nrows=1).columns[0]

# Print title and dataframe
print(GHCN_v4_title)
df_GHCN_v4

Annual Temperature Anomalies (deg C) GHCNv4/ERSSTv5 vs. 2007-2016


,Year,Glob,N.Hemi,S.Hemi,24N-90N,24S-24N,90S-24S,64N-90N,44N-64N,24N-44N,EQU-24N,24S-EQU,44S-24S,64S-44S,90S-64S
0,2003,-0.099,-0.144,-0.055,-0.233,0.007,-0.108,-0.563,-0.102,-0.211,-0.010,0.024,-0.177,0.066,-0.250
1,2004,-0.184,-0.232,-0.135,-0.349,-0.081,-0.155,-1.597,-0.132,-0.087,-0.058,-0.105,-0.141,0.058,-0.677
2,2005,-0.041,-0.035,-0.047,-0.060,-0.012,-0.062,-0.081,0.141,-0.181,0.001,-0.025,-0.130,0.012,0.016
3,2006,-0.081,-0.074,-0.087,-0.088,-0.076,-0.081,-0.326,-0.024,-0.054,-0.054,-0.097,-0.108,0.021,-0.213
4,2007,-0.054,-0.038,-0.069,0.052,-0.168,-0.006,0.036,0.238,-0.062,-0.173,-0.163,-0.124,-0.101,0.595
5,2008,-0.175,-0.209,-0.142,-0.159,-0.256,-0.086,-0.530,-0.006,-0.136,-0.286,-0.225,-0.097,-0.086,-0.042
6,2009,-0.062,-0.167,0.042,-0.297,0.047,0.028,-0.645,-0.481,-0.070,0.029,0.064,-0.021,-0.024,0.311
7,2010,0.006,0.009,0.003,-0.051,0.054,-0.001,0.156,-0.218,-0.013,0.098,0.009,0.010,0.030,-0.119
8,2011,-0.108,-0.147,-0.068,-0.090,-0.260,0.079,0.277,-0.146,-0.173,-0.232,-0.289,0.018,0.008,0.463
9,2012,-0.070,-0.078,-0.062,-0.034,-0.106,-0.058,0.141,-0.155,-0.014,-0.144,-0.068,-0.062,0.020,-0.215
